# 판다스 데이터프레임 불러오기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

이 튜토리얼은 판다스 라이브러리의 데이터프레임을 텐서플로우의 'tf.data.Dataset'으로 불러오는 방법에 관한 예시를 제공합니다.  

이 튜토리얼은 클리블랜드(Cleveland) 심장병 재단에서 제공하는 [데이터셋](https://archive.ics.uci.edu/ml/datasets/heart+Disease)을 사용합니다. 이 CSV 파일에는 수 백 개의 행이 있습니다. 각각의 행은 환자를 의미하고, 각 열은 환자가 가진 특성들을 의미합니다. 이러한 정보들을 그 환자가 심장 질환을 가지고 있는지 없는지 예측하는 이진 분류 작업(binary classification task)에 사용할 것입니다. 



## 판다스를 이용해 데이터 읽어오기

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import pandas as pd
import tensorflow as tf

심장 데이터셋을 갖고 있는 CSV 파일을 다운로드 합니다.




In [0]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

판다스를 사용하여 CSV 파일을 읽습니다.

In [0]:
df = pd.read_csv(csv_file)

In [0]:
df.head()

In [0]:
df.dtypes

데이터프레임의 `객체`인 `thal` 열을 이산 수치값(discrete numerical value)으로 변환합니다.

In [0]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [0]:
df.head()

## 'tf.data.Dataset'을 이용하여 데이터 불러오기

판다스의 데이터프레임에서 값을 읽어오기 위하여 'tf.data.Dataset.from_tensor_slices'를 사용합니다.
 
'tf.data.Dataset'을 사용하는 이점 중 하나는 간단하고, 효율적으로 데이터 파이프라인을 구축 할 수 있다는 것입니다. 이것에 대해서는 [데이터 불러오기 가이드](https://www.tensorflow.org/guide/data)(loading data guide)를 참고하여 더 알아볼 수 있습니다.

In [0]:
target = df.pop('target')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [0]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

`pd.Series`가 `__array__` 프로토콜을 사용하기 때문에, 그것은 분명하게 `np.array`나 `tf.Tensor`를 사용할 수 있는 거의 어느 곳에서나 사용 될 수 있습니다.

In [0]:
tf.constant(df['thal'])

데이터 셋을 섞고, 배치(batch)합니다.

In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## 모델을 만들고, 훈련하기

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

## 특성 열(feature column) 다른 방식으로 구성하기

딕셔너리를 모델의 인풋으로 전달하는 것은 전처리 하고, 그것들을 [functional api](../../guide/keras/functional.ipynb)를 사용하여 쌓아 만든'tf.keras.layers.Input'의 matching dictionary를 만드는 것 만큼이나 쉽습니다. 이것을 [특성 열](../keras/feature_columns.ipynb)(feature column)의 대안으로 사용할 수 있습니다. 

In [0]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

 `pd.DataFrame`이 `tf.data`와 함께 사용될 때, 그 열의 구조를 보존할 수 있는 가장 쉬운 방법은 `pd.DataFrame`을 `dict`로 변환하고, 그 딕셔너리를 자르는(slice) 것입니다.

In [0]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [0]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

In [0]:
model_func.fit(dict_slices, epochs=15)